# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902067


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:14,  2.68s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:50,  2.02s/it]

Rendering models:  23%|██▎       | 7/30 [00:06<00:33,  1.44s/it]

Rendering models:  30%|███       | 9/30 [00:07<00:21,  1.02s/it]

...

Rendering models:  80%|████████  | 24/30 [00:07<00:01,  5.87it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  6.94it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  8.48it/s]

Rendering models: 100%|██████████| 30/30 [00:08<00:00,  9.13it/s]

Thedunkmasta                          0.001451
garburgers                            0.017568
MrMoney5956                           0.001180
not-logged-in-2967ebd21102cdfdf1ce    0.349011
minichick                             0.000243
Linda_J._Berkel                       0.001564
JEBala                                0.054016
not-logged-in-ea8ed61a7274cfb6a43b    0.007650
not-logged-in-ea8ed61a7274cfb6a43b    0.082962
SohaKawtharani                        0.001204
Paddy_van_CH                          0.000247
not-logged-in-a0164d241f30411408f0    0.000255
ComeMyChild                           0.130278
ericmalani                            0.011259
coyotes54                             0.000260
BirdLeaf                              0.000361
not-logged-in-f82a6d15a1dafa9bbfae    0.000187
DarioB                                0.001124
Lavadude                              0.169201
Mr_DMan                               0.001023
not-logged-in-704ca07c4131cfda3ea2    0.010561
equidad1     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())